# Goal
The goal is to use a simple model to classify x-ray images in Keras, the notebook how to use the ```flow_from_dataframe``` to deal with messier datasets

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread
import cv2

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from glob import glob
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
!ls ../input/sample/sample

In [ ]:
!mkdir ~/.keras
!mkdir ~/.keras/models
!cp ../input/keraspretrainedmodels/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keraspretrainedmodels/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
#!cp ../input/keraspretrainedmodels/keras-pretrained-models/resnet50* ~/.keras/models/

import os

cache_dir = os.path.expanduser(os.path.join('~', '.keras'))
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

# Create symbolic links for trained models.
# Thanks to Lem Lordje Ko for the idea
# https://www.kaggle.com/lemonkoala/pretrained-keras-models-symlinked-not-copied
models_symlink = os.path.join(cache_dir, 'models')
if not os.path.exists(models_symlink):
    os.symlink('/kaggle/input/keras-pretrained-models/', models_symlink)

In [ ]:
# ../input/
PATH = os.path.abspath(os.path.join('..', 'input'))

# ../input/sample/images/
SOURCE_IMAGES = os.path.join(PATH, "sample", "sample","images")

# ../input/sample/images/*.png
images = glob(os.path.join(SOURCE_IMAGES, "*.png"))

# Load labels
all_xray_df = pd.read_csv('../input/sample/sample_labels.csv')
all_image_paths = {os.path.basename(x): x for x in images}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_xray_df['Patient Age'] = all_xray_df['Patient Age'].map(lambda x: int(x[:-1]))
all_xray_df['Patient Age'] = np.clip(all_xray_df['Patient Age'], 5, 100)
all_xray_df.sample(3)

# Preprocessing Labels
Here we take the labels and make them into a more clear format. The primary step is to see the distribution of findings and then to convert them to simple binary labels

In [ ]:
label_counts = all_xray_df['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
from itertools import chain
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        all_xray_df[c_label] = all_xray_df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
all_xray_df.head(5)

In [ ]:
all_xray_df[['Patient Age', 'Infiltration']].hist(figsize = (10, 5))

In [ ]:
###Balance the whole data by droping the oversampling set
positive_cases = np.sum(all_xray_df['Infiltration']==1)//2
oversample_factor = 4 # maximum number of cases in negative group so it isn't super rare
all_xray_df = all_xray_df.groupby(['Patient Gender', 'Infiltration']).apply(lambda x: x.sample(min(oversample_factor*positive_cases, x.shape[0]), replace = False)).reset_index(drop = True)

print(more_balanced_df['Infiltration'].value_counts())
all_xray_df[['Patient Age', 'Infiltration']].hist(figsize = (10, 5))

# Prepare Training Data
Here we split the data into training and validation sets and create a single vector (disease_vec) with the 0/1 outputs for the disease status (what the model will try and predict)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(all_xray_df, 
                                   test_size = 0.2, 
                                   random_state = 2018,
                                   stratify = all_xray_df[['Patient Gender','Patient Gender']])
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

In [ ]:
###Balance the data in the training set
new_train_df = train_df.groupby(['Infiltration']).apply(lambda x: x.sample(1900, replace = True)).reset_index(drop = True)
print('New Data Size:', new_train_df.shape[0], 'Old Size:', train_df.shape[0])
new_train_df[['Infiltration', 'Patient Age']].hist(figsize = (10, 5))

# Create Data Generators
Here we make the data generators for loading and randomly transforming images

In [ ]:
#from keras.applications.mobilenet import MobileNet
from keras.applications.resnet50 import ResNet50, preprocess_input
#from keras.applications.vgg16 import VGG16, preprocess_input
#from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop

In [ ]:

IMG_SIZE = (484, 484)
core_idg = ImageDataGenerator(preprocessing_function=preprocess_input,
                              samplewise_center=False, 
                              samplewise_std_normalization=False, 
                              horizontal_flip= False,  
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=10, 
                              shear_range = 0.01,
                              fill_mode = 'nearest',
                              zoom_range=0.15)
val_idg = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [ ]:
train_gen = flow_from_dataframe(core_idg, new_train_df, 
                             path_col = 'path',
                            y_col = 'Infiltration', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 256)

valid_gen = flow_from_dataframe(core_idg, valid_df, 
                             path_col = 'path',
                            y_col = 'Infiltration', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 384) # we can use much larger batches for evaluation
# used a fixed dataset for evaluating the algorithm
test_X, test_Y = next(flow_from_dataframe(core_idg, valid_df, 
                             path_col = 'path',
                            y_col = 'Infiltration', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 384)) # one big batch

In [ ]:
t_x, t_y = next(train_gen)
fig, m_axs = plt.subplots(2, 4, figsize = (16, 8))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone')
    c_ax.set_title('%s' % ('Infiltration' if c_y>0.5 else 'Else'))
    c_ax.axis('off')

# Create a simple model
Here we make a simple model to train using MobileNet as a base and then adding a GAP layer (Flatten could also be added), dropout, and a fully-connected layer to calculate specific features

In [ ]:

from keras import regularizers
base_model = ResNet50(input_shape =  (IMG_SIZE[0],IMG_SIZE[1],3), include_top = False, weights = 'imagenet')
#base_model.trainable = False
for layer in base_model.layers[:-4]:
    layer.trainable= False
for layer in base_model.layers:
    if 'bn' in layer.name:
        layer.trainable = False
    print(layer,layer.trainable)
print (len(base_model.layers))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(512, activation = 'relu',kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = "adam", loss = 'binary_crossentropy',metrics = ['binary_accuracy', 'mae'])
model.summary()

from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda, AvgPool2D
from keras.models import Model
from keras.layers import BatchNormalization

base_model = ResNet50(input_shape =  (IMG_SIZE[0],IMG_SIZE[1],3), 
                                 include_top = False, weights = 'imagenet')
#base_model.trainable = False
for layer in base_model.layers[:-4]:
    layer.trainable= False
for layer in base_model.layers:
    if 'bn' in layer.name:
        layer.trainable = False
    print(layer,layer.trainable)

pt_features = Input(base_model.get_output_shape_at(0)[1:], name = 'feature_input')
pt_depth = base_model.get_output_shape_at(0)[-1]
#print(base_model.get_output_shape_at(0)[-1])
bn_features = BatchNormalization()(pt_features)

# here we do an attention mechanism to turn pixels in the GAP on an off
attn_layer = Conv2D(128, kernel_size = (1,1), padding = 'same', activation = 'elu')(bn_features)
attn_layer = Conv2D(32, kernel_size = (1,1), padding = 'same', activation = 'elu')(attn_layer)
attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'elu')(attn_layer)
attn_layer = AvgPool2D((2,2), strides = (1,1), padding = 'same')(attn_layer) # smooth results
attn_layer = Conv2D(1, 
                    kernel_size = (1,1), 
                    padding = 'valid', 
                    activation = 'sigmoid')(attn_layer)
# fan it out to all of the channels
up_c2_w = np.ones((1, 1, 1, pt_depth))
up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
               activation = 'linear', use_bias = False, weights = [up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(attn_layer)

mask_features = multiply([attn_layer, bn_features])
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model
gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
gap_dr = Dropout(0.5)(gap)
dr_steps = Dropout(0.5)(Dense(128, activation = 'elu')(gap_dr))
out_layer = Dense(1, activation = 'sigmoid')(dr_steps)

attn_model = Model(inputs = [pt_features], outputs = [out_layer], name = 'attention_model')

attn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy'])

attn_model.summary()    

from keras.models import Sequential
from keras.optimizers import Adam
model = Sequential(name = 'combined_model')
#base_model.trainable = False
model.add(base_model)
model.add(attn_model)
model.compile(optimizer = Adam(lr = 1e-3), loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('xray_class')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5)
callbacks_list = [checkpoint, early]

# First Round
Here we do a first round of training to get a few initial low hanging fruit results

In [ ]:
train_gen.batch_size = 24
valid_gen.batch_size = 32
model.fit_generator(train_gen, 
                    steps_per_epoch=train_gen.samples//train_gen.batch_size,
                    validation_data = valid_gen,
                    validation_steps = valid_gen.samples//valid_gen.batch_size,
                    epochs = 1, 
                    callbacks = callbacks_list,
                    workers = 3)

# Continued Training
Now we do a much longer training process to see how the results improve

In [ ]:
history = model.fit_generator(train_gen, 
                              steps_per_epoch = train_gen.samples//train_gen.batch_size,
                              validation_data =  valid_gen, 
                              validation_steps = valid_gen.samples//valid_gen.batch_size,
                              epochs = 15, 
                              callbacks = callbacks_list)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.ylim(0.8,0.96)
plt.legend(['train', 'test'], loc='upper left')
plt.show()
fig.savefig('model_accuracy.png', dpi=300)
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
fig.savefig('model_loss.png', dpi=300)

In [ ]:
# load the best weights
model.load_weights(weight_path)

In [ ]:
pred_Y = model.predict(test_X, batch_size = 32, verbose = True)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, _ = roc_curve(test_Y, pred_Y)
fig, ax1 = plt.subplots(1,1, figsize = (5, 5), dpi = 250)
ax1.plot(fpr, tpr, 'b.-', label = 'RestNet-Model (AUC:%2.2f)' % roc_auc_score(test_Y, pred_Y))
ax1.plot(fpr, fpr, 'k-', label = 'Random Guessing')
ax1.legend(loc = 4)
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate');
fig.savefig('roc.pdf')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
plt.matshow(confusion_matrix(test_Y, pred_Y>0.5))
print(classification_report(test_Y, pred_Y>0.5, target_names = ['Else', 'Infiltration']))